In [19]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import requests
from bs4 import BeautifulSoup
import re

In [53]:
def openchrome():
    option = webdriver.ChromeOptions()
    option.add_argument(r'user-data-dir=C:\Users\hujia\AppData\Local\Google\Chrome\User Data')
    option.add_argument('--headless')
    driver = webdriver.Chrome(options = option)
    return driver

In [151]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
from sklearn.tree import DecisionTreeClassifier
import re
import time
import requests
from selenium import webdriver
import numpy as np
import pandas as pd

def OfferLocalBrowser(headless=False):
    '''
    这个函数用于提供自动登录大众点评的Chrome浏览器
    :param headless: 是否使用无头Chrome
    :return: 返回浏览器对象
    '''
    option = webdriver.ChromeOptions()
    option.add_argument(r'user-data-dir=C:\Users\hujia\AppData\Local\Google\Chrome\User Data')
    if headless:
        option.add_argument('--headless')
    browser = webdriver.Chrome(options=option)

    return browser

def CollectDataset(targetUrl,low=10,high=15,page=10,refreshTime=8):
    '''
    
    :param targetUrl: 传入可翻页的任意商铺评论页面地址
    :param low: 设置随机睡眠防ban的随机整数下限
    :param high: 设置随机睡眠防ban的随机整数上限
    :param page: 设置最大翻页次数
    :param refreshTime: 设置每个页面重复刷新的时间
    :return: 返回收集到的汉字列表和编码列表
    '''


    '''初始化用于存放所有采集到的样本词和对应的样本词编码的列表,CL用于存放所有编码，WL用于存放所有词，二者顺序一一对应'''
    CL,WL = [],[]
    AL = []

    browser = OfferLocalBrowser(headless=True)
    for p in tqdm(range(1,page+1)):
        pal = []
        for r in range(refreshTime):
            '''访问目标网页'''
            html = browser.get(url=targetUrl.format(p))
            print(targetUrl.format(p))
            if '3s 未完成验证，请重试。' in str(browser.page_source):
                ii = input()

            '''将原始网页内容解码'''
            html = browser.page_source
            '''解析网页内容'''
            obj = BeautifulSoup(html,'lxml')
            '''提取评论部分内容以方便之后对评论汉字和SVG图像对应编码的提取'''
            raw_comment = obj.find_all('div',{'class':'review-words Hide'})
            '''初始化列表容器以有顺序地存放符合汉字或SVG标签格式的内容'''
            base_Comment = []
            '''利用正则提取符合汉字内容规则的元素'''
            firstList = re.findall('(<span class="[a-z0-9]+">)|([\u4e00-\u9fa5]{1})',str(raw_comment))
            '''构造该页面中长度守恒的评论片段列表'''
            actualList = []
            '''按顺序将所有汉字片段和<span>标签片段拼接在一起'''
            for i in range(len(firstList)):
                for j in range(2):
                    if firstList[i][j] != '':
                        actualList.append(firstList[i][j])
            '''打印当前界面所有评论片段的长度'''
            print(len(actualList))
            
            '''在每个页面的第一次访问时初始化汉字列表和编码列表'''
            if r == 0:
                wordList = ['' for i in range(len(actualList))]
                codeList = ['' for i in range(len(actualList))]
            '''将actualList中粗糙的<span>片段清洗成纯粹的编码片段，汉字部分则原封不定保留，并分别更新wordList和codeList'''
            for index in range(len(actualList)):
                if '<' in actualList[index]:
                    codeList[index] = re.findall('class="([a-z0-9]+)"',actualList[index])[0]
                    actualList[index] = re.findall('class="([a-z0-9]+)"',actualList[index])[0]

                else:
                    wordList[index] = actualList[index]
            '''随机睡眠防ban'''
            pal.append(actualList)
            time.sleep(np.random.randint(low,high))
        '''将结束重复采集的当前页面中发现的所有汉字-编码对应规则列表与先前的规则列表合并'''
        AL.append(pal)
        CL.extend(codeList)
        WL.extend(wordList)
        print('总列表长度：{}'.format(len(CL)))

    browser.quit()

    return WL,CL,AL

In [153]:
url = r'http://www.dianping.com/shop/79608412/review_all/p{}?queryType=sortType&queryVal=latest'
wl,cl,al = CollectDataset(targetUrl=url)





























  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

http://www.dianping.com/shop/79608412/review_all/p1?queryType=sortType&queryVal=latest
664
http://www.dianping.com/shop/79608412/review_all/p1?queryType=sortType&queryVal=latest
664
http://www.dianping.com/shop/79608412/review_all/p1?queryType=sortType&queryVal=latest
664
http://www.dianping.com/shop/79608412/review_all/p1?queryType=sortType&queryVal=latest
664
http://www.dianping.com/shop/79608412/review_all/p1?queryType=sortType&queryVal=latest
664
http://www.dianping.com/shop/79608412/review_all/p1?queryType=sortType&queryVal=latest
664
http://www.dianping.com/shop/79608412/review_all/p1?queryType=sortType&queryVal=latest
664
http://www.dianping.com/shop/79608412/review_all/p1?queryType=sortType&queryVal=latest
664
总列表长度：664






























 10%|████████▏                                                                         | 1/10 [01:54<17:07, 114.20s/it]

http://www.dianping.com/shop/79608412/review_all/p2?queryType=sortType&queryVal=latest
1209
http://www.dianping.com/shop/79608412/review_all/p2?queryType=sortType&queryVal=latest
1209
http://www.dianping.com/shop/79608412/review_all/p2?queryType=sortType&queryVal=latest
1209
http://www.dianping.com/shop/79608412/review_all/p2?queryType=sortType&queryVal=latest
1209
http://www.dianping.com/shop/79608412/review_all/p2?queryType=sortType&queryVal=latest
1209
http://www.dianping.com/shop/79608412/review_all/p2?queryType=sortType&queryVal=latest
1209
http://www.dianping.com/shop/79608412/review_all/p2?queryType=sortType&queryVal=latest
1209
http://www.dianping.com/shop/79608412/review_all/p2?queryType=sortType&queryVal=latest
1209
总列表长度：1873






























 20%|████████████████▍                                                                 | 2/10 [04:01<15:45, 118.21s/it]

...





























 80%|█████████████████████████████████████████████████████████████████▌                | 8/10 [16:03<03:58, 119.45s/it]

http://www.dianping.com/shop/79608412/review_all/p9?queryType=sortType&queryVal=latest
1915
http://www.dianping.com/shop/79608412/review_all/p9?queryType=sortType&queryVal=latest
1915
http://www.dianping.com/shop/79608412/review_all/p9?queryType=sortType&queryVal=latest
1915
http://www.dianping.com/shop/79608412/review_all/p9?queryType=sortType&queryVal=latest
1915
http://www.dianping.com/shop/79608412/review_all/p9?queryType=sortType&queryVal=latest
1915
http://www.dianping.com/shop/79608412/review_all/p9?queryType=sortType&queryVal=latest
1915
http://www.dianping.com/shop/79608412/review_all/p9?queryType=sortType&queryVal=latest
1915
http://www.dianping.com/shop/79608412/review_all/p9?queryType=sortType&queryVal=latest
1915
总列表长度：11307






























 90%|█████████████████████████████████████████████████████████████████████████▊        | 9/10 [17:59<01:58, 118.42s/it]

http://www.dianping.com/shop/79608412/review_all/p10?queryType=sortType&queryVal=latest
971
http://www.dianping.com/shop/79608412/review_all/p10?queryType=sortType&queryVal=latest
971
http://www.dianping.com/shop/79608412/review_all/p10?queryType=sortType&queryVal=latest
971
http://www.dianping.com/shop/79608412/review_all/p10?queryType=sortType&queryVal=latest
971
http://www.dianping.com/shop/79608412/review_all/p10?queryType=sortType&queryVal=latest
971
http://www.dianping.com/shop/79608412/review_all/p10?queryType=sortType&queryVal=latest
971
http://www.dianping.com/shop/79608412/review_all/p10?queryType=sortType&queryVal=latest
971
http://www.dianping.com/shop/79608412/review_all/p10?queryType=sortType&queryVal=latest
971
总列表长度：12278






























100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [20:00<00:00, 119.26s/it]





























In [156]:
columns = ['a','b','c','d','e','f','g','h']
ala = pd.DataFrame(al,columns = columns)
ala['a'][0]

['jqjmk',
 '名',
 '字',
 'jqwcv',
 '引',
 'jq01s',
 'jq84i',
 '家',
 '店',
 '日',
 '式',
 '居',
 '酒',
 'jqdr1',
 'jq01s',
 'jq9js',
 'jqkfk',
 '座',
 '位',
 'jqoyh',
 '小',
 '桌',
 '和',
 '类',
 '似',
 '于',
 'jqgmz',
 '台',
 'jq01s',
 '位',
 'jqzbc',
 'jqkfk',
 '局',
 'jqb3i',
 '得',
 'jqoyh',
 'jqdm1',
 'jqeyy',
 '凑',
 '招',
 '牌',
 'jq9o0',
 '碎',
 'jqg78',
 'jqdm1',
 'jqtfi',
 'jq84i',
 '份',
 '主',
 '食',
 'jqtb1',
 '起',
 '来',
 '每',
 'jq84i',
 '口',
 'jqsu8',
 'jqoyh',
 'jq9o0',
 '肉',
 '碎',
 '米',
 'jqg78',
 'jqb4w',
 'jq72a',
 '泉',
 '蛋',
 'jq01s',
 '混',
 '合',
 '特',
 '别',
 'jq01s',
 '巧',
 'jqzxm',
 '推',
 '荐',
 '哦',
 'jqm7j',
 '果',
 '大',
 'jq5ys',
 '卷',
 'jqmxn',
 'jq01s',
 '敲',
 'jqd0x',
 'jqtb1',
 'jqm7j',
 '果',
 '和',
 'jqlqo',
 'jqlqo',
 'jq01s',
 'jq5ys',
 '配',
 '合',
 '在',
 'jq84i',
 '起',
 'jqmxn',
 'jq01s',
 '很',
 'jqd0x',
 'jqtb1',
 '芥',
 '末',
 '章',
 '鱼',
 '日',
 '料',
 '必',
 'jqdm1',
 '小',
 '菜',
 '之',
 'jq84i',
 '收',
 '起',
 '评',
 '论',
 '文',
 '字',
 '收',
 '起',
 '评',
 '论',
 '在',
 '望',
 '京',
 '旁',
 '边',
 'jq

In [157]:
t = ala['a'][0]
s = [None for i in range(len(t))]
for i,item in enumerate(t):
    if u'\u4e00' <= item[0] <= u'\u9fff':
        s[i] = item
    else:
        for j,col in enumerate(columns):
            if u'\u4e00' <= ala[col][0][i][0] <= u'\u9fff':
                s[i] = ala[col][0][i]
print(s)        

['被', '名', '字', '吸', '引', '的', '一', '家', '店', '日', '式', '居', '酒', '屋', '的', '风', '格', '座', '位', '有', '小', '桌', '和', '类', '似', '于', '吧', '台', '的', '位', '置', '格', '局', '显', '得', '有', '点', '紧', '凑', '招', '牌', '鸡', '碎', '饭', '点', '了', '一', '份', '主', '食', '吃', '起', '来', '每', '一', '口', '都', '有', '鸡', '肉', '碎', '米', '饭', '与', '温', '泉', '蛋', '的', '混', '合', '特', '别', '的', '巧', '妙', '推', '荐', '哦', '芒', '果', '大', '虾', '卷', '真', '的', '敲', '好', '吃', '芒', '果', '和', '脆', '脆', '的', '虾', '配', '合', '在', '一', '起', '真', '的', '很', '好', '吃', '芥', '末', '章', '鱼', '日', '料', '必', '点', '小', '菜', '之', '一', '收', '起', '评', '论', '文', '字', '收', '起', '评', '论', '在', '望', '京', '旁', '边', None, '胡', '同', '里', '一', '家', '日', '料', '讲', '真', '好', '看', None, '外', '国', '妹', '子', '很', '多', '点', '了', '一', '份', '人', '套', '餐', '三', '个', '人', '吃', None, '很', '饱', '鳗', '鱼', '寿', '司', '很', '正', '宗', '一', '口', '入', '嘴', '还', '有', '套', '餐', '里', '带', None, '地', '狱', '乌', '冬', '面', '有', '点', '麻', '辣', '烫', None, '味', '道', '套', '餐', '里',

In [163]:
ss = list(filter(None,s))
''.join(ss)

'被名字吸引的一家店日式居酒屋的风格座位有小桌和类似于吧台的位置格局显得有点紧凑招牌鸡碎饭点了一份主食吃起来每一口都有鸡肉碎米饭与温泉蛋的混合特别的巧妙推荐哦芒果大虾卷真的敲好吃芒果和脆脆的虾配合在一起真的很好吃芥末章鱼日料必点小菜之一收起评论文字收起评论在望京旁边胡同里一家日料讲真好看外国妹子很多点了一份人套餐三个人吃很饱鳗鱼寿司很正宗一口入嘴还有套餐里带地狱乌冬面有点麻辣烫味道套餐里牛肉小火锅也是为这个夏天前寒冷带来一丝温暖吧温泉蛋牛油果纯纯蛋白质呀很棒收起评论文字收起评论麒麟社里的日料店少这家却是少有的工作日到八点左右还需要等位点了一个寿喜锅一个寿司和一盘盐烤拼盘寿喜锅的味道错会像很多地方的一样上来吃的时候很咸但是鸡蛋要单加钱寿司中规中矩现没有留下什么印象盐烤拼盘菜如其名微咸收起评论文字收起评论人很多导致服务不太好环境其实一般那边有几家很大的这家不算出众食物味道还可以招牌寿司很棒因为老公不喜欢寿喜锅一个人点了一份牛肉锅味道还不错还点了招牌鸡碎饭吃起来很棒蛋液很可爱啊收起评论文字收起评论下午半开餐钟可以先进去等着两个人了样其中三样都很不错寿喜锅必比较正宗的好吃招牌鸡碎饭也是必推荐鲸一特制卷他们家最好吃的寿司推荐到这里两个人的量会剩一烤鸡肉难吃别收起评论文字收起评论麒麟社好吃的简直太多了看哪家都好吃结果真的差强意吃完了都没有太大的惊喜个感觉这样的小饭馆儿就是互相模仿没有点儿自己的特色或者是他有别没有的东西随大溜儿就行了不难吃服务什么都还行吧收起评论文字收起评论'

In [69]:
%%time
url = r'http://www.dianping.com/shop/79608412/review_all'
N = 1
AL = pd.DataFrame()

for n in range(N):
    browser = openchrome()
    html = browser.get(url)
    html = browser.page_source

    obj = BeautifulSoup(html,'lxml')
    '''提取评论部分内容以方便之后对评论汉字和SVG图像对应编码的提取'''
    raw_comment = obj.find_all('div',{'class':'review-words Hide'})
    '''初始化列表容器以有顺序地存放符合汉字或SVG标签格式的内容'''
    base_Comment = []
    '''利用正则提取符合汉字内容规则的元素'''
    #firstList = re.findall('(<span class="[a-z0-9]+">)|([\u4e00-\u9fa5]{1})|([，,。*！？.～:\【】·「」★])',str(raw_comment))
    firstList = re.findall('(<span class="[a-z0-9]+">)|([\u4e00-\u9fa5]{1})',str(raw_comment))
    browser.quit()

    print(firstList)
    
#print(firstList)

[('<span class="xw6tf">', ''), ('', '于'), ('', '麒'), ('', '麟'), ('<span class="xw8qp">', ''), ('', '的'), ('', '一'), ('<span class="xw5wz">', ''), ('', '日'), ('<span class="xwj8q">', ''), ('<span class="xwpxf">', ''), ('<span class="xwpxf">', ''), ('', '里'), ('<span class="xwox2">', ''), ('<span class="xwfqt">', ''), ('<span class="xwqjq">', ''), ('', '人'), ('<span class="xw6tf">', ''), ('<span class="xwaa0">', ''), ('', '人'), ('<span class="xw6tf">', ''), ('<span class="xwlo9">', ''), ('', '合'), ('', '约'), ('<span class="xwi56">', ''), ('', '的'), ('', '地'), ('', '方'), ('', '旁'), ('', '边'), ('', '坐'), ('', '的'), ('<span class="xwgku">', ''), ('<span class="xwfqt">', ''), ('<span class="xwkfo">', ''), ('', '侣'), ('<span class="xw8vp">', ''), ('<span class="xw11n">', ''), ('', '顺'), ('<span class="xw01s">', ''), ('<span class="xw3fc">', ''), ('', '瞄'), ('', '周'), ('', '围'), ('', '脑'), ('', '补'), ('', '各'), ('', '种'), ('', '剧'), ('<span class="xwkfo">', ''), ('<span class="xw41z">', ''), (


Wall time: 14.9 s


In [38]:
t1 = BeautifulSoup(str(raw_comment))
t2 = t1.find_all('span')
#t1.span.attrs['class']
#t2[0].get('class')
t1.get_text()

'[\n于麒麟一家日料，里是两四，适约会地，旁边全是情侣，饭顺便瞄周围，补各情。。。最爱拉啦，虽国内鸡品敢保，偶尔事吧。鸡碎饭，果，锅一般（感自己在家也能做来），烧串儿！烤干\n                            \n\n\t\t                            收起评论\n\n\n, \n                            午选这，这王先生提意，接开车过，这其实他，他总说，饭肚感觉，别中那烤，觉得烤，难得他配合一，慰。也，望京为看女，大，务还情，多忙碌，分内外，随，错，一糙，硬头皮说，超大厚切三文，一感觉实，火锅甜，，地，最欢依油果海沙和芥末章。准，评价，过清还等去本再。\n                            \n\n\t\t                            收起评论\n\n\n, \n                            对食太友好了「温泉拌油」芥末味刚好冲，油金成熟，的溏流！「招牌鸡」来溏拌吃，，鸡超香！「煮」噢的被煮的的，胡也的。甜，分也足。「鹅寿」太香了，鹅入即化腻，吃还来，包阻止了。💰吃了百出🙋整调味重。他~下次找陪起来吃\n                            \n\n\t\t                            收起评论\n\n\n, \n愧望京TOP3日店，得还错哦，食材鲜，格还实惠，比，饭前队。荐鹅肝，做非常好，这很考验功底！稍慎。果3-4还荐下锅，毕竟热门，分量很，果2再些估会。PS嘤嘤嘤，然发橙V变灰，月然4评吗，恢复才好！\n                            \n\n\t\t                            收起评论\n\n\n, \n                            💫✨打卡鲸堂✨💫吃什么，评里搜圈最后相中牌，看评论等位去并，直接位置是是节假日关系。很，和品都更贴近于日式简餐。愧是牌，配流温泉蛋～呀～个滋～唯缺是量，个姑娘吃完，再份又多种。园拉也错～般般，无无过。\n                            \n\n\t\t                            收起评论\n\n\n, \

In [165]:
def CreateXandY(wordList,codeList,cssUrl,SvgUrl):
    '''
    这个函数用于传入朴素的汉字列表、编码列表、CSS页面地址，SVG页面地址来输出规整的numpy多维数组格式的自变量X，以及标签Y
    :param wordList: 汉字列表
    :param codeList: 编码列表
    :param cssUrl: CSS页面地址
    :param SvgUrl: SVG页面地址
    :return: 返回自变量X，因变量Y
    '''

    def GetSvgWordIpx(SvgUrl=SvgUrl):
        '''
        这个函数用于爬取SVG页面，并返回所需内容
        :param SvgUrl: SVG页面地址
        :return: 单个汉字为键，上面所列四个属性为汉字键对应嵌套的字典中对应值的字典文件
        '''

        '''访问SVG页面'''
        SvgWord = requests.get(SvgUrl).content.decode('utf-8')

        '''初始化汉字-候选因变量字典'''
        Svg2Label = {}

        '''提取SVG页面中所有汉字所在的text标签内容列表，每个列表对应页面中一行文字'''
        rawList = re.findall('[\u4e00-\u9fa5]+', SvgWord)

        '''抽取每个汉字及其对应的四个候选因变量'''
        for row in range(len(rawList)):
            wordPreList = re.findall('[\u4e00-\u9fa5]{1}', rawList[row])
            for word in wordPreList:
                Svg2Label[word] = {
                    'RowIndex': [],
                    'ColIndex': []
                }
                Svg2Label[word]['RowIndex'] = row + 1
                Svg2Label[word]['ColIndex'] = wordPreList.index(word) + 1

        return Svg2Label

    '''访问CSS页面'''
    CodeWithIpx = requests.get(cssUrl).content.decode()

    '''初始化编码-px值字典'''
    code2ipx = {}

    '''初始化针对样本数据的编码-汉字字典'''
    code2word = {}

    '''从样本中抽取采集到的确切的汉字-编码关系'''
    for code, word in tqdm(zip(codeList, wordList)):
        if code != '' and word != '':
            code2ipx[code] = re.search(
                '.%s{background:-(.*?).0px -(.*?).0px;}' % code, CodeWithIpx).groups()
            code2word[code] = word

    Svg2Label = GetSvgWordIpx()

    '''生成自变量和因变量'''

    X = []
    for key, value in code2ipx.items():
        X.append([int(value[0]), int(value[1])])

    X = np.array(X)

    Y = []
    for key, value in code2ipx.items():
        Y.append([Svg2Label[code2word[key]]['RowIndex'],
                  Svg2Label[code2word[key]]['ColIndex']])

    Y = np.array(Y)

    return X,Y,Svg2Label,CodeWithIpx

In [169]:
cssurl = r'http://s3plus.meituan.net/v1/mss_0a06a471f9514fc79c981b5466f56b91/svgtextcss/1d445720fa9be88f46a6ced32dbcf691.css'
svgurl = r'http://s3plus.meituan.net/v1/mss_0a06a471f9514fc79c981b5466f56b91/svgtextcss/ee1e5ab954c8b12330e0768fff7fe9f5.svg'
x,y,svg2label,code2ipx = CreateXandY(wl,cl,cssurl,svgurl)





























0it [00:00, ?it/s]





























AttributeError: 'NoneType' object has no attribute 'groups'

In [172]:
cl

['jqjmk',
 'jqpvs',
 'jq5no',
 'jqwcv',
 'jqz3s',
 'jq01s',
 'jq84i',
 'jqvgh',
 'jqwyn',
 'jqj9k',
 'jqgxv',
 'jqvyz',
 'jq7ua',
 'jqdr1',
 'jq01s',
 'jq9js',
 'jqkfk',
 'jqvg6',
 'jq4jk',
 'jqoyh',
 'jqkzj',
 'jq58d',
 'jq8i5',
 'jqkck',
 'jqmoy',
 'jq1m1',
 'jqgmz',
 'jq5xh',
 'jq01s',
 'jq4jk',
 'jqzbc',
 'jqkfk',
 'jqo56',
 'jqb3i',
 'jqsok',
 'jqoyh',
 'jqdm1',
 'jqeyy',
 'jq50q',
 'jqaso',
 'jqna5',
 'jq9o0',
 'jqp5r',
 'jqg78',
 'jqdm1',
 'jqtfi',
 'jq84i',
 'jq5lt',
 'jqhfd',
 'jql8e',
 'jqtb1',
 'jqixo',
 'jq3by',
 'jq4cm',
 'jq84i',
 'jq77z',
 'jqsu8',
 'jqoyh',
 'jq9o0',
 'jq9a8',
 'jqp5r',
 '',
 'jqg78',
 'jqb4w',
 'jq72a',
 'jqezg',
 'jqrz9',
 'jq01s',
 'jqr8o',
 'jqqvg',
 'jqdt7',
 'jqxsu',
 'jq01s',
 'jqxkn',
 'jqzxm',
 'jqm8o',
 'jqcep',
 '',
 'jqm7j',
 'jqfat',
 'jq23q',
 'jq5ys',
 'jq1fq',
 'jqmxn',
 'jq01s',
 'jq2cs',
 'jqd0x',
 'jqtb1',
 'jqm7j',
 'jqfat',
 'jq8i5',
 'jqlqo',
 'jqlqo',
 'jq01s',
 'jq5ys',
 'jqhqi',
 'jqqvg',
 'jqt1l',
 'jq84i',
 'jqixo',
 'jqmxn',


In [57]:
svgu = r'http://s3plus.meituan.net/v1/mss_0a06a471f9514fc79c981b5466f56b91/svgtextcss/ac7cb937d8c4811777bee4c950ab09fc.svg'
t2 = requests.get(svgu).content.decode('utf-8')
t3 = re.findall('[\u4e00-\u9fa5]+', t2)
t5 = {}
for row in range(len(t3)):
    t4 = re.findall('[\u4e00-\u9fa5]{1}', t3[row])
    for word in t4:
        t5[word] = {
            'RowIndex': [],
            'ColIndex': []
        }
        t5[word]['RowIndex'] = row + 1
        t5[word]['ColIndex'] = t4.index(word) + 1
        
print(t5)

{'吩': {'RowIndex': 1, 'ColIndex': 1}, '和': {'RowIndex': 1, 'ColIndex': 2}, '第': {'RowIndex': 1, 'ColIndex': 3}, '试': {'RowIndex': 1, 'ColIndex': 4}, '斯': {'RowIndex': 1, 'ColIndex': 5}, '榴': {'RowIndex': 1, 'ColIndex': 6}, '廉': {'RowIndex': 1, 'ColIndex': 7}, '顷': {'RowIndex': 1, 'ColIndex': 8}, '组': {'RowIndex': 1, 'ColIndex': 9}, '较': {'RowIndex': 1, 'ColIndex': 10}, '姥': {'RowIndex': 1, 'ColIndex': 11}, '三': {'RowIndex': 1, 'ColIndex': 12}, '落': {'RowIndex': 1, 'ColIndex': 13}, '盲': {'RowIndex': 1, 'ColIndex': 14}, '雁': {'RowIndex': 1, 'ColIndex': 15}, '所': {'RowIndex': 1, 'ColIndex': 16}, '局': {'RowIndex': 1, 'ColIndex': 17}, '肩': {'RowIndex': 1, 'ColIndex': 18}, '卸': {'RowIndex': 1, 'ColIndex': 19}, '换': {'RowIndex': 1, 'ColIndex': 20}, '燥': {'RowIndex': 1, 'ColIndex': 21}, '督': {'RowIndex': 1, 'ColIndex': 22}, '谈': {'RowIndex': 1, 'ColIndex': 23}, '抄': {'RowIndex': 1, 'ColIndex': 24}, '冲': {'RowIndex': 1, 'ColIndex': 25}, '黄': {'RowIndex': 1, 'ColIndex': 26}, '驻': {'RowIndex': 1,

In [12]:
def GetModels(X,Y):
    '''
    
    :param X: 因变量
    :param Y: 自变量
    :return: 用于预测行下标的模型1和预测列下标的模型2
    '''
    '''这个模型的因变量为对应汉字的行下标'''
    model1 = DecisionTreeClassifier().fit(X, Y[:, 0])

    '''这个模型的因变量是对应汉字的列下标'''
    model2 = DecisionTreeClassifier().fit(X, Y[:, 1])
    return model1,model2

def Translate(s,baseDF,model1,model2):
    '''
    这个函数用于对汉字和SVG标签格式混杂的字符串进行预测解码
    :param s: 待解码的字符串
    :param baseDF: 存放所有汉字与其行列下标的数据框
    :param model1: 模型1
    :param model2: 模型2
    :return: 预测解码结果
    '''
    result = ''
    for ele in s:
        for u in range(2):
            if ele[u] != '' and '<' in ele[u]:
                row_ = model1.predict(np.array(
                    [int(re.search('.%s{background:-(.*?).0px -(.*?).0px;}' % re.search('<span class="([a-z0-9]+)">',ele[u]).group(1), CodeWithIpx).groups()[i]) for i in
                     range(2)]).reshape(1, -1))
                col_ = model2.predict(np.array(
                    [int(re.search('.%s{background:-(.*?).0px -(.*?).0px;}' % re.search('<span class="([a-z0-9]+)">',ele[u]).group(1), CodeWithIpx).groups()[i]) for i in
                     range(2)]).reshape(1, -1))


                answer = baseDF['字符'][(baseDF['Row'] == row_.tolist()[0]) & (baseDF['Col'] == col_.tolist()[0])].tolist()[0]

                result += answer
            else:
                result += ele[u]

    return result

baseDF = pd.DataFrame({'字符': [key for key in svg2label.keys()],
                                       'Row': [svg2label[key]['RowIndex'] for key in svg2label.keys()],
                                       'Col': [svg2label[key]['ColIndex'] for key in svg2label.keys()]})



In [17]:
baseDF

,字符,Row,Col
0,挂,1,1
1,握,1,2
2,拣,1,3
3,熄,1,4
4,坡,1,5
5,厕,1,6
6,浸,1,7
7,踢,1,8
8,蕉,1,9
9,积,1,10
